In [1]:
%load_ext autoreload
%autoreload 2

import tensorflow as tf
import warnings
warnings.filterwarnings('ignore')
import sys

sys.path.append('./../../')
import matplotlib.pyplot as plt
from batchflow.opensets import MNIST
from batchflow.models.tf import TFModel
from batchflow import Pipeline, L, F, V, D, B, DatasetIndex, Dataset, ImagesBatch, Config

import numpy as np

In [26]:
mnist = MNIST(batch_class=ImagesBatch)

DownloadingDownloading http://yann.lecun.com/exdb/mnist/train-labels-idx1-ubyte.gz
http://yann.lecun.com/exdb/mnist/train-images-idx3-ubyte.gz
Downloaded t10k-labels-idx1-ubyte.gz
Extracting /tmp/t10k-labels-idx1-ubyte.gz
Downloaded train-labels-idx1-ubyte.gz
Extracting /tmp/train-labels-idx1-ubyte.gz
Downloaded t10k-images-idx3-ubyte.gz
Extracting /tmp/t10k-images-idx3-ubyte.gz
Downloaded train-images-idx3-ubyte.gz
Extracting /tmp/train-images-idx3-ubyte.gz


In [27]:
decay_d = ('exp', {'learning_rate': 0.001,
                   'decay_steps': 150,
                   'decay_rate': 0.96})

model_config = {
    'inputs': dict(images={'shape': (28, 28, 1)}, labels={'classes': 10}),
    'initial_block': {'layout': 'cna'*2,
                      'filters': [6]*2, 'kernel_size': [3]*2,
                      'inputs': 'images'},
    'body': {'layout': 'nca nca',
             'filters': [16]*2, 'kernel_size': [3]*2},
    'head': {'layout': 'Pfa', 'units': 10},
    'loss': 'ce',
    'decay': decay_d,
    'train_steps': {'ALL': {'optimizer': 'RMSProp', 'decay': decay_d},
                    'BODY': {'optimizer': 'Adam', 'scope': 'body', 'loss': 'ce'},
                    'HEAD': {'optimizer': 'Adagrad', 'scope': 'head', 'decay': decay_d},
                    'CUSTOM': {'optimizer': 'Adam', 'scope': '-initial_block/layer-0'},
                    'RRRRRRR': {'optimizer': 'Adam', 'scope': ['initial_block/layer-0', 'initial_block/layer-1']}},
}

In [24]:
TFModel(model_config)


ALL
<tf.Variable 'TFModel/initial_block/layer-0/conv2d/kernel:0' shape=(3, 3, 1, 6) dtype=float32_ref>
<tf.Variable 'TFModel/initial_block/layer-0/conv2d/bias:0' shape=(6,) dtype=float32_ref>
<tf.Variable 'TFModel/initial_block/layer-1/batch_normalization/gamma:0' shape=(6,) dtype=float32_ref>
<tf.Variable 'TFModel/initial_block/layer-1/batch_normalization/beta:0' shape=(6,) dtype=float32_ref>
<tf.Variable 'TFModel/initial_block/layer-3/conv2d/kernel:0' shape=(3, 3, 6, 6) dtype=float32_ref>
<tf.Variable 'TFModel/initial_block/layer-3/conv2d/bias:0' shape=(6,) dtype=float32_ref>
<tf.Variable 'TFModel/initial_block/layer-4/batch_normalization/gamma:0' shape=(6,) dtype=float32_ref>
<tf.Variable 'TFModel/initial_block/layer-4/batch_normalization/beta:0' shape=(6,) dtype=float32_ref>
<tf.Variable 'TFModel/body/layer-0/batch_normalization/gamma:0' shape=(6,) dtype=float32_ref>
<tf.Variable 'TFModel/body/layer-0/batch_normalization/beta:0' shape=(6,) dtype=float32_ref>
<tf.Variable 'TFModel/

In [32]:
train_pipeline = (mnist.train.p
                 .init_variable('loss_history', init_on_each_run=list)
                 .init_variable('current_loss')
                 .to_array()
                 .multiply(multiplier=1/255., preserve_type=False)
                 .init_model('dynamic', TFModel, 'conv', config=model_config)
                 .to_array()
                 .train_model('conv', fetches='loss_RRRRRRR', images=B('images'), labels=B('labels'), 
                              train_mode='RRRRRRR',
                              save_to=V('current_loss'))
                 .update_variable('loss_history', 
                                  V('current_loss'), 
                                  mode='a')
                  )

In [33]:
%%time
n_b = train_pipeline.next_batch(256, n_epochs=None)
# n_b = train_pipeline.run(256, n_epochs=2, bar=True)


ALL
<tf.Variable 'TFModel/initial_block/layer-0/conv2d/kernel:0' shape=(3, 3, 1, 6) dtype=float32_ref>
<tf.Variable 'TFModel/initial_block/layer-0/conv2d/bias:0' shape=(6,) dtype=float32_ref>
<tf.Variable 'TFModel/initial_block/layer-1/batch_normalization/gamma:0' shape=(6,) dtype=float32_ref>
<tf.Variable 'TFModel/initial_block/layer-1/batch_normalization/beta:0' shape=(6,) dtype=float32_ref>
<tf.Variable 'TFModel/initial_block/layer-3/conv2d/kernel:0' shape=(3, 3, 6, 6) dtype=float32_ref>
<tf.Variable 'TFModel/initial_block/layer-3/conv2d/bias:0' shape=(6,) dtype=float32_ref>
<tf.Variable 'TFModel/initial_block/layer-4/batch_normalization/gamma:0' shape=(6,) dtype=float32_ref>
<tf.Variable 'TFModel/initial_block/layer-4/batch_normalization/beta:0' shape=(6,) dtype=float32_ref>
<tf.Variable 'TFModel/body/layer-0/batch_normalization/gamma:0' shape=(6,) dtype=float32_ref>
<tf.Variable 'TFModel/body/layer-0/batch_normalization/beta:0' shape=(6,) dtype=float32_ref>
<tf.Variable 'TFModel/

In [ ]:
%%time
_ = train_pipeline.run(batch_size=256, n_epochs=1, bar=True)

In [ ]:
plt.plot(train_pipeline.get_variable("loss_history"))
plt.xlabel("Iterations"), plt.ylabel("Loss")
plt.show()